In [ ]:
# =============
# CARGA DE LIBRERIAS

from astropy.io import fits
import glob
import numpy as np
import os
import pickle
import random
import sys

In [ ]:
# =============
# INTERPOLACION DE LOS ESPECTROS Y GUARDADO EN FORMATO PICKLE

folder_path = r'spectrums'
batch_size = 10000  # Numero de archivos a procesar por lote
index_file = "extra/batch_index.txt"

# Verificar que el archivo batch_index.txt exista
if not os.path.exists(index_file):
    print(f"Error: El archivo {index_file} no existe. Deteniendo la ejecución.")
    sys.exit(1)

# Intentar leer y convertir el contenido a un numero entero
with open(index_file, "r") as f:
    content = f.read().strip()
    try:
        batch_index = int(content)
    except ValueError:
        print(f"Error: El archivo {index_file} no contiene un número entero válido. Deteniendo la ejecución.")
        sys.exit(1)

def expand_points(wavelength, flux, target_count=5000):
    # Convertir a listas para facilitar las inserciones
    wl = list(wavelength)
    fl = list(flux)
    
    # Calcular las diferencias absolutas entre puntos consecutivos
    diffs = [abs(fl[i+1] - fl[i]) for i in range(len(fl)-1)]
    # Obtener los indices ordenados de mayor a menor diferencia
    sorted_indices = sorted(range(len(diffs)), key=lambda i: diffs[i], reverse=True)
    
    # Insertar nuevos puntos utilizando los indices ordenados
    while len(wl) < target_count:
        # Se recorre la lista de indices en orden descendente para evitar problemas con el reordenamiento
        for idx in sorted_indices:
            if len(wl) >= target_count:
                break
            # Calcular la interpolacion lineal entre el punto idx y el siguiente
            new_wl = (wl[idx] + wl[idx+1]) / 2
            new_fl = (fl[idx] + fl[idx+1]) / 2
            # Insertar el nuevo punto en la posicion correspondiente
            wl.insert(idx+1, new_wl)
            fl.insert(idx+1, new_fl)
    
    return np.array(wl), np.array(fl)

# Obtener la lista inicial de archivos FITS
files = [f for f in os.listdir(folder_path) if f.endswith('.fits')]

while files:
    # Seleccionar aleatoriamente el lote actual (si quedan menos de batch_size, se toman todos)
    if len(files) < batch_size:
        current_batch = files.copy()
    else:
        current_batch = random.sample(files, batch_size)
    
    # Diccionario de almacenamiento para el lote actual
    spectra_data = {}
    i = 0

    for filename in current_batch:
        file_path = os.path.join(folder_path, filename)
        try:
            with fits.open(file_path) as hdul:
                # Verifica que el archivo tenga las extensiones esperadas
                if len(hdul) > 2:
                    flux_data = hdul[1].data["flux"]            # Datos de flujo
                    loglam_data = hdul[1].data["loglam"]        # Datos de log(lambda)
                    wavelength_data = 10 ** loglam_data         # Convertir log(lambda) a longitud de onda
                    redshift = hdul[2].data["Z"][0]             # Extraer redshift
                    
                    # Aplicar la interpolacion para obtener 5000 puntos
                    interp_wavelength, interp_flux = expand_points(wavelength_data, flux_data, target_count=5000)
                    
                    # Guardar los datos
                    spectra_data[filename] = {
                        "wavelength": interp_wavelength,
                        "flux": interp_flux,
                        "redshift": redshift
                    }
                    
                    i += 1
                    if i % 1000 == 0:
                        print(f"Procesado {filename} ({i})")
        except Exception as e:
            print(f"Error procesando {filename}: {e}")
    
    print(f"Se procesaron {len(spectra_data)} archivos FITS en el lote {batch_index}.")
    
    # Guardar el diccionario en un archivo pickle
    output_file = f'data/spectra_data_complete{batch_index}.pkl'
    with open(output_file, 'wb') as f:
        pickle.dump(spectra_data, f)
    print(f"Datos guardados en {output_file}")
    
    # Guardar la lista de archivos procesados en este lote
    batch_list_file = f'extra/batch_files_list{batch_index}.pkl'
    with open(batch_list_file, 'wb') as f:
        pickle.dump(current_batch, f)
    print(f"Lista de archivos del lote {batch_index} guardada en {batch_list_file}")
    
    # Eliminar los nuevos procesados de files
    files = [f for f in files if f not in current_batch]
    
    # Actualizar el archivo batch_index.txt para la siguiente iteracion
    batch_index += 1
    with open(index_file, "w") as f:
        f.write(str(batch_index))
    
print("Se han procesado todos los archivos de la carpeta.")

In [ ]:
# =============
# ELIMINADOR DE LOS .fits YA PROCESADOS

pickle_folder = 'extra'
spectrums_folder = r'spectrums'

# Obtener la lista de archivos pickle que comienzan con "batch_files_list" y terminan con ".pkl"
pickle_files = [f for f in os.listdir(pickle_folder) if f.startswith('batch_files_list') and f.endswith('.pkl')]

# Recorrer cada archivo pickle
for pickle_file in pickle_files:
    pickle_path = os.path.join(pickle_folder, pickle_file)
    print(f"Procesando {pickle_path}...")
    try:
        with open(pickle_path, 'rb') as f:
            file_list = pickle.load(f)
    except Exception as e:
        print(f"Error abriendo {pickle_path}: {e}")
        continue
    # Recorrer la lista de nombres y eliminar los archivos correspondientes en la carpeta spectrums
    for filename in file_list:
        file_path = os.path.join(spectrums_folder, filename)
        if os.path.exists(file_path):
            try:
                os.remove(file_path)
                print(f"Eliminado: {file_path}")
            except Exception as e:
                print(f"Error eliminando {file_path}: {e}")
        else:
            print(f"Archivo no encontrado: {file_path}")

print("Proceso de eliminación completado.")

In [ ]:
# =============
# CONVERSION A NUMPY ARRAYS

# Buscar todos los archivos que comiencen con un cierto prefijo en la carpeta "data"
data_files = glob.glob(os.path.join('data', 'spectra_data_complete*.pkl'))
data_files = sorted(data_files)
print(f"Se encontraron {len(data_files)} archivos pickle.")

# Contar el numero total de espectros
total_spectra = 0
for file in data_files:
    with open(file, 'rb') as f:
        data_part = pickle.load(f)
    total_spectra += len(data_part)
print(f"Total de espectros: {total_spectra}")

# Prealojar archivos memmap para flux, wavelength y redshift en la carpeta 'data'
flux_mmap_path = os.path.join('data', 'spectra_data_complete_flux.dat')
wavelength_mmap_path = os.path.join('data', 'spectra_data_complete_wavelength.dat')
redshift_mmap_path = os.path.join('data', 'spectra_data_complete_redshift.dat')

num_points = 5000
flux_mmap = np.memmap(flux_mmap_path, dtype='float32', mode='w+', shape=(total_spectra, num_points))
wavelength_mmap = np.memmap(wavelength_mmap_path, dtype='float32', mode='w+', shape=(total_spectra, num_points))
redshift_mmap = np.memmap(redshift_mmap_path, dtype='float32', mode='w+', shape=(total_spectra,))

# Cargar datos de los archivos pickle y escribirlos en los memmaps
current_index = 0
for file in data_files:
    with open(file, 'rb') as f:
        data_part = pickle.load(f)
    print(f"Procesando {file} con {len(data_part)} espectros...")
    for key in data_part:
        espectro = data_part[key]
        flux = espectro["flux"]              # Array de 5000 puntos
        wavelength = espectro["wavelength"]  # Array de 5000 puntos
        redshift = espectro["redshift"]      # Valor escalar
        
        # Almacenar en el memmap
        flux_mmap[current_index, :] = flux
        wavelength_mmap[current_index, :] = wavelength
        redshift_mmap[current_index] = redshift
        
        current_index += 1

# Asegurar que todos los cambios se escriban en disco
flux_mmap.flush()
wavelength_mmap.flush()
redshift_mmap.flush()

print("Se han guardado los datos en archivos memmap dentro de la carpeta 'data'.")

In [ ]:
# =============
# INTERPOLACION DE LOS ESPECTROS Y GUARDADO EN FORMATO PICKLE (VARIANTE PARA EL NED)

folder_path = r'spectrums NASA'
batch_size = 100000
index_file = "extra/batch_index.txt"

# Verificar existencia de batch_index.txt
if not os.path.exists(index_file):
    print(f"Error: El archivo {index_file} no existe. Deteniendo la ejecución.")
    sys.exit(1)

# Leer y validar batch_index
with open(index_file, "r") as f:
    content = f.read().strip()
    try:
        batch_index = int(content)
    except ValueError:
        print(f"Error: El archivo {index_file} no contiene un número entero válido. Deteniendo la ejecución.")
        sys.exit(1)

def expand_points(wavelength, flux, target_count=5000):
    wl = list(wavelength)
    fl = list(flux)
    # Diferencias absolutas entre puntos consecutivos
    diffs = [abs(fl[i+1] - fl[i]) for i in range(len(fl)-1)]
    sorted_indices = sorted(range(len(diffs)), key=lambda i: diffs[i], reverse=True)
    # Insertar hasta alcanzar target_count
    while len(wl) < target_count:
        for idx in sorted_indices:
            if len(wl) >= target_count:
                break
            new_wl = (wl[idx] + wl[idx+1]) / 2
            new_fl = (fl[idx] + fl[idx+1]) / 2
            wl.insert(idx+1, new_wl)
            fl.insert(idx+1, new_fl)
    return np.array(wl), np.array(fl)

# Lista de archivos FITS
files = [f for f in os.listdir(folder_path) if f.endswith('.fits')]

while files:
    current_batch = files.copy() if len(files) < batch_size else random.sample(files, batch_size)
    spectra_data = {}
    processed = 0

    for filename in current_batch:
        file_path = os.path.join(folder_path, filename)
        try:
            with fits.open(file_path) as hdul:
                try:
                    data    = hdul[0].data      # shape (5,3856)
                    header0 = hdul[0].header

                    # Construir vector de longitudes de onda (log‑linear)
                    n_pix  = header0['NAXIS1']
                    coeff0 = header0['COEFF0']
                    coeff1 = header0['COEFF1']
                    crpix1 = header0.get('CRPIX1', 1)
                    pix    = np.arange(n_pix)
                    wave   = 10**(coeff0 + coeff1 * (pix + 1 - crpix1))
                    flux  = data[1, :]
                    redshift   = header0.get('Z')

                    # Limpiar NaN antes de expandir
                    mask = np.isfinite(flux)
                    wavelength_data = wave[mask]
                    flux_data = flux[mask]

                except Exception as e:
                    continue

                # Interpolar hasta 5000 puntos en flux
                interp_wavelength = wavelength_data
                interp_flux = flux_data
                # Aplicar expand_points en bucle
                while interp_flux.shape[0] < 5000:
                    interp_wavelength, interp_flux = expand_points(interp_wavelength, interp_flux, target_count=5000)

                spectra_data[filename] = {
                    "wavelength": interp_wavelength,
                    "flux": interp_flux,
                    "redshift": redshift
                }
                processed += 1
                if processed % 1000 == 0:
                    print(f"Procesado {processed} espectros en este lote.")
        except Exception as e:
            print(f"Error procesando {filename}: {e}")  # sigue con el siguiente archivo

    print(f"Se procesaron {len(spectra_data)} archivos FITS en el lote {batch_index}.")

    # Guardar resultados
    output_file = f'data/spectra_data_NASA{batch_index}.pkl'
    with open(output_file, 'wb') as f:
        pickle.dump(spectra_data, f)
    print(f"Datos guardados en {output_file}")

    batch_list_file = f'extra/batch_files_list{batch_index}.pkl'
    with open(batch_list_file, 'wb') as f:
        pickle.dump(current_batch, f)
    print(f"Lista de archivos del lote {batch_index} guardada en {batch_list_file}")

    # Actualizar lista y batch_index
    files = [f for f in files if f not in current_batch]
    batch_index += 1
    with open(index_file, "w") as f:
        f.write(str(batch_index))

print("Se han procesado todos los archivos de la carpeta.")